In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import os

/home/joowan/anaconda3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# # specify path to data
# path2data = 'dataset/STL'

# # if not exists the path, make the directory
# if not os.path.exists(path2data):
#     os.mkdir(path2data)

# # load dataset
# train_ds = datasets.STL10(path2data, split='train', download=True, transform=transforms.ToTensor())
# val_ds = datasets.STL10(path2data, split='test', download=True, transform=transforms.ToTensor())

# print(len(train_ds))
# print(len(val_ds))


# # define transformation
# transformation = transforms.Compose([
#                     transforms.ToTensor(),
#                     transforms.Resize(224),
#                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
# ])

# # apply transformation to dataset
# train_ds.transform = transformation
# val_ds.transform = transformation

# # make dataloade
# train_dl = DataLoader(train_ds, batch_size=64, shuffle=True,num_workers=8)
# val_dl = DataLoader(val_ds, batch_size=16, shuffle=True,num_workers=2)

In [5]:
path2data = 'dataset/STL'

# load dataset
train_ds = datasets.STL10(path2data, split='train', download=False, transform=transforms.ToTensor())
val_ds = datasets.STL10(path2data, split='test', download=False, transform=transforms.ToTensor())

print(len(train_ds))
print(len(val_ds))


# define transformation
transformation = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Resize(224),
                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# apply transformation to dataset
train_ds.transform = transformation
val_ds.transform = transformation

# make dataloade
train_dl = DataLoader(train_ds, batch_size=64, shuffle=True,num_workers=8)
val_dl = DataLoader(val_ds, batch_size=16, shuffle=True,num_workers=2)

5000
8000


In [6]:
val_ds

Dataset STL10
    Number of datapoints: 8000
    Root location: dataset/STL
    Split: test
    StandardTransform
Transform: ToTensor()

In [7]:
import torch.nn as nn
from models import swin_transformer
import torch.optim as optim
from tqdm import tqdm

device='cuda:0'
net=swin_transformer.SwinTransformer(num_classes=10)
net=net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.01)


[Warning] Fused window process have not been installed. Please refer to get_started.md for installation.


In [8]:

train_loss=[]
val_loss=[]
train_acc=[]
val_acc=[]
max_epoch=100

for epoch in range(max_epoch):
    print('*'*30)
    print(f'{epoch+1}/{max_epoch}')
    
    tq=tqdm(train_dl, ncols=80, smoothing=0, bar_format='train: {desc}|{bar}{r_bar}')
    vtq=tqdm(val_dl, ncols=80, smoothing=0, bar_format='val: {desc}|{bar}{r_bar}')
    
    
    running_loss = 0.0
    corrects=0
    for i, data in tqdm(enumerate(tq)):
        # [inputs, labels]의 목록인 data로부터 입력을 받은 후;
        inputs, labels = data
        inputs=inputs.to(device)
        labels=labels.to(device)
       

        # 변화도(Gradient) 매개변수를 0으로 만들고
        optimizer.zero_grad()

        # 순전파 + 역전파 + 최적화를 한 후
        output = net(inputs)
        print(output)
        loss = criterion(output, labels)
        pred=output.argmax(1,keepdim=True)
        corrects+=pred.eq(labels.view_as(pred)).sum().item()
        
        loss.backward()
        optimizer.step()

        # 통계를 출력합니다.
        running_loss += loss.item()
        
    
    loss=running_loss/len(train_dl.dataset)
    acc=corrects/len(train_dl.dataset)
    train_loss.append(loss)
    train_acc.append(acc)
    
    print(f'train_loss: {loss}, train_accuray:{acc*100}%')
    
    
    
    running_loss = 0.0
    corrects=0
    for i, data in tqdm(enumerate(vtq)):
        inputs, labels = data
        inputs=inputs.to(device)
        labels=labels.to(device)
       

        # 변화도(Gradient) 매개변수를 0으로 만들고
        optimizer.zero_grad()

        # 순전파 + 역전파 + 최적화를 한 후
        output = net(inputs)
        loss = criterion(output, labels)
        pred=output.argmax(1,keepdim=True)
        corrects+=pred.eq(labels.view_as(pred)).sum().item()
        
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
    
    loss=running_loss/len(val_dl.dataset)
    acc=corrects/len(val_dl.dataset)
    val_loss.append(loss)
    val_acc.append(acc)
    
    print(f'val_loss: {loss}, val_accuracy{acc*100}%')
    running_loss = 0.0







******************************
1/100


train: |                                                 | 0/79 [00:00<?, ?it/s]

train: |▌                                        | 1/79 [00:00<01:07,  1.15it/s]
train: |█                                        | 2/79 [00:01<00:42,  1.83it/s]
train: |█▌                                       | 3/79 [00:01<00:34,  2.20it/s]
train: |██                                       | 4/79 [00:01<00:29,  2.51it/s]
train: |██▌                                      | 5/79 [00:01<00:27,  2.73it/s]
train: |███                                      | 6/79 [00:02<00:25,  2.91it/s]
train: |███▋                                     | 7/79 [00:02<00:23,  3.05it/s]
train: |████▏                                    | 8/79 [00:02<00:22,  3.17it/s]
train: |████▋                                    | 9/79 [00:02<00:21,  3.26it/s]
train: |█████                                   | 10/79 [00:02<00:20,  3.35it/s]
train: |█████▌                                  | 11/79 [00:03<00:19,  3.42it/s]
train: |██████             

train_loss: 0.05238460307121277, train_accuray:0.0978


213it [00:14, 14.56it/s]
val: |█████████████████                       | 213/500 [00:33<00:45,  6.37it/s]


KeyboardInterrupt: 